# Embeddings Module

### Install necessary modules

In [1]:
%pip install -U -e ..
%pip install -e ..
%pip install pydantic==2.9.2
%pip install nest_asyncio

Obtaining file:///Users/jose/Repos/MINE-DD
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached pydantic_core-2.33.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
Using cached pydantic_core-2.33.1-cp312-cp312-macosx_11_0_arm64.whl (1.9 MB)
  Building editable for minedd (pyproject.toml) ... done
  Created wheel for minedd: filename=minedd-0.1.0-0.editable-py3-none-any.whl size=13035 sha256=31791f26e9e1b10bf77681f59e60988028fb0dbb8a10c4179d7abf3c7d437198
  Stored in directory: /private/var/folders/79/zf67ls7520x9m4mj7nx6q07w0000gp/T/pip-ephem-wheel-cache-iquykvnj/wheels/c7/bf/1e/e7790fb2ba7cdeeb0fd2dc5eb82103f3cd4a31d3c897656e15
Successfully built minedd
  Attempting uninstall: pydantic-core
    

In [2]:
# This is needed to run asyncio code in Jupyter notebooks
# without getting "RuntimeError: This event loop is already running"
import nest_asyncio
nest_asyncio.apply()

### Load Embeddings object

In [3]:
from minedd.embeddings import Embeddings

embeddings = Embeddings(
    output_embeddings_path="my-embeddings.pkl",
)
embeddings

Embeddings(output_embeddings_path=my-embeddings.pkl, docs=None)

### Load papers for processing

In [4]:
from pathlib import Path
PAPERS_DIRECTORY = Path.home() / "papers_minedd/"
pdf_file_list = embeddings.prepare_papers(PAPERS_DIRECTORY)
print(len(pdf_file_list))
# Check the first 10 files
pdf_file_list[:10]

11


['Ambient temperature and age-related notified Campylobacter infection in Israel_ A 12-year time series study.pdf',
 'A comparison of weather variables linked to infectious disease patterns using laboratory addresses and patient residence addresses.pdf',
 '.DS_Store',
 '_i_Campylobacter__i_ Monitoring in German Broiler Flocks_ An Explorative Time Series Analysis.pdf',
 'A time series analysis of the relationship of ambient temperature and common bacterial enteric infections in two Canadian provinces.pdf',
 'A Bayesian spatio-temporal framework to identify outbreaks and examine environmental and social risk factors for infectious diseases monitored by routine surveillance.pdf',
 'A time-series study of the association of rainfall_ relative humidity and ambient temperature with hospitalizations for rotavirus and norovirus infection among children in Hong Kong.pdf',
 'Weather Variability and the Incidence of Cryptosporidiosis_ Comparison of Time Series Poisson Regression and SARIMA Models

### Define paperQA settings

In [5]:
from paperqa.settings import Settings, AgentSettings, ParsingSettings

MODEL = "ollama/llama3.2"
EMBEDDING = "ollama/mxbai-embed-large:latest"

"""Configure settings for paperqa."""
local_llm_config = {
    "model_list": [
        {
            "model_name": MODEL,
            "litellm_params": {
                "model": MODEL,
                "api_base": "http://0.0.0.0:11434",
            },
            "answer": {
                "evidence_k": 40,
                "evidence_detailed_citations": True,
                "evidence_summary_length": "about 100 words",
                "answer_max_sources": 10,
                "answer_length": "about 600 words, but can be longer",
                "max_concurrent_requests": 10,
                "answer_filter_extra_background": False
            },
            "parsing": {
                "use_doc_details": True
            }
        }
    ]
}

settings = Settings(
    llm=MODEL,
    llm_config=local_llm_config,
    summary_llm=MODEL,
    summary_llm_config=local_llm_config,
    paper_directory=str(PAPERS_DIRECTORY),
    embedding=EMBEDDING,
    agent=AgentSettings(
        agent_llm=MODEL,
        agent_llm_config=local_llm_config,
        return_paper_metadata=True
    ),
    parsing=ParsingSettings(
        chunk_size=2500,
        overlap=250
    )
)

### Create/Load Embeddings and save in PKL

In [6]:
import os
if os.path.exists("my-embeddings.pkl"):
    embeddings.load_existing_embeddings("my-embeddings.pkl")
else:
    embeddings.process_papers(settings, PAPERS_DIRECTORY, pdf_file_list)
    print("Embeddings created and saved to my-embeddings.pkl")

Creating new Docs object.


  0%|          | 0/11 [00:00<?, ?it/s]/opt/miniconda3/envs/poopqa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
SEMANTIC_SCHOLAR_API_KEY environment variable not set. Semantic Scholar API rate limits may apply.
CROSSREF_MAILTO environment variable not set. Crossref API rate limits may apply.
CROSSREF_API_KEY environment variable not set. Crossref API rate limits may apply.
Metadata not found for Ambient temperature and age-related notified Campylobacter infection in TIsrael: A 12-year time series study in SemanticScholarProvider.
Request to CrossrefProvider for Ambient temperature and age-related notified Campylobacter infection in TIsrael: A 12-year time series study timed out.
  9%|▉         | 1/11 [00:19<03:12, 19.22s/it]

Correctly loaded Ambient temperature and age-related notified Campylobacter infection in Israel_ A 12-year time series study.pdf


Failed to parse all of title, DOI, and authors from the ParsingSettings.structured_citation_prompt's response {title: str, authors: list[str], doi: str}, consider using a manifest file or specifying a different citation prompt.
 18%|█▊        | 2/11 [00:44<03:27, 23.02s/it]

Correctly loaded A comparison of weather variables linked to infectious disease patterns using laboratory addresses and patient residence addresses.pdf
Could not read .DS_Store: ParsedText.content must be a `list`, not <class 'str'>.


...

Correctly loaded _i_Campylobacter__i_epidemiology_ a descriptive study reviewing 1 million cases in England and Wales between 1989 and 2011.pdf


 91%|█████████ | 10/11 [03:03<00:20, 20.03s/it]

Correctly loaded Weather and notified Campylobacter infections in temperate and sub-tropical regions of Australia_ An ecological study.pdf


Failed to parse all of title, DOI, and authors from the ParsingSettings.structured_citation_prompt's response {"title": None, "authors": [], "doi": None}, consider using a manifest file or specifying a different citation prompt.
100%|██████████| 11/11 [03:21<00:00, 18.32s/it]

Correctly loaded Are hospitalizations for rotavirus gastroenteritis associated with meteorologic factors_.pdf
Docs object saved to my-embeddings.pkl
Embeddings created and saved to my-embeddings.pkl


### Inspect Documents Object

In [7]:
detail_df = embeddings.get_docs_details()
detail_df

,doc_key,key,docname,title,authors,year,journal,volume,pages,doi,url,file_location,citation_count,source_quality
0,ef5fb575eb743e9e2ddf54e5e0c9743e,rosenbergaUnknownyearambienttemperatureand,Ambient temperature and age-related notified C...,Ambient temperature and age-related notified C...,"[Alina Rosenberga, Miriam Weinberger, Shlomit ...",NaN,None,None,None,None,None,None,NaN,NaN
1,5e1061700ab7312396d7b7fd503e8c2f,None,A comparison of weather variables linked to in...,None,None,NaN,None,None,None,None,None,None,NaN,NaN
2,82429582fe1ae21b,s.2009campylobactermonitoringin,s.2009campylobactermonitoringin,Campylobacter Monitoring in German Broiler Flo...,"[Hartnack, S., Doherr, M. G., Alter, T., Touto...",2009.0,Zoonoses and Public Health,56,None,10.1111/j.1863-2378.2008.01184.x,,None,31.0,2.0
3,bf02d614f82ce026fa308eeeaf192053,None,A time series analysis of the relationship of ...,None,None,NaN,None,None,None,None,None,None,NaN,NaN
4,b924171c933be72d,aparna2018abayesianspatiotemporal,aparna2018abayesianspatiotemporal,A Bayesian spatio-temporal framework to identi...,"[Lal, Aparna, Marshall, Jonathan, Benschop, Ja...",2018.0,Spatial and Spatio-temporal Epidemiology,25,39-48,10.1016/j.sste.2017.10.004,https://doi.org/10.1016/j.sste.2017.10.004,None,13.0,1.0
5,b6a6137315ccac14,pin2018atimeseriesstudy,pin2018atimeseriesstudy,A time-series study of the association of rain...,"[Wang, Pin, William B. Goggins, Emily Y.Y. Chan]",2018.0,The Science of the total environment,643,\n414-422\n,10.1016/j.scitotenv.2018.06.189,,None,69.0,-1.0
6,bed463ca3b0dd66e,hu2007weathervariabilityand,hu2007weathervariabilityand,Weather Variability and the Incidence of Crypt...,"[Wenbiao Hu, Shilu Tong, Kerrie Mengeresen, De...",2007.0,Annals of Epidemiology,17,679-688,10.1016/j.annepidem.2007.03.020,https://doi.org/10.1016/j.annepidem.2007.03.020,None,82.0,1.0
7,aff6d96d1f88ea6d15ef934239c39e02,None,_i_Campylobacter__i_epidemiology_ a descriptiv...,None,None,NaN,None,None,None,None,None,None,NaN,NaN
8,e2e754915d56cacf7780abef4f1fdf7b,None,Weather and notified Campylobacter infections ...,None,None,NaN,None,None,None,None,None,None,NaN,NaN
9,b696c2a7a6e6c7df3d1d59c628c1b725,None,Are hospitalizations for rotavirus gastroenter...,None,None,NaN,None,None,None,None,None,None,NaN,NaN


In [8]:
print(len(embeddings.docs.texts))
for doc in embeddings.docs.texts:
    print(len(doc.text))

219
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2160
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2351
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2026
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2059
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2094
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
1637
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
1340
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
1296
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2